In [53]:
import scrapy
from scrapy.crawler import CrawlerProcess

import csv
import pandas as pd
import numpy as np

class AmazonReviewsSpider(scrapy.Spider):
    name = 'amazon_reviews'
    allowed_domains = ['amazon.com']
    
    # Страница с отзывами, которую парсим
    start_urls = ['https://www.amazon.es/CREATE-THERA-Cafetera-monodosis-semiautom%C3%A1tica/product-reviews/B0BSH8CYN8?ie=UTF8&reviewerType=avp_only_reviews']

    def parse(self, response):
        # Открываем CSV файл для записи данных
        with open('amazon_reviews.csv', 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['title', 'rating', 'body', 'author', 'date']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if csvfile.tell() == 0:
                writer.writeheader()

            # Извлекаем данные об отзывах
            reviews = response.css('.review')

            for review in reviews:
                writer.writerow({
                    'title': review.css('.review-title').css('span::text').get(),
                    'rating': review.css('.review-rating').css('span::text').get(),
                    'body': review.css('.review-text').css('span::text').get(),
                    'author': review.css('.a-profile-name::text').get(),
                    'date': review.css('.review-date::text').get(),
                })
        
        # Находим ссылку на следующую страницу с отзывами и продолжаем парсинг
        next_page = response.css('.a-last a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

# Запуск процесса 
if __name__ == "__main__":
    process = CrawlerProcess(settings={ # Это эмуляция браузера, без нее может прилететь БАН
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'FEEDS': {
            'amazon_reviews.csv': {
                'format': 'csv',
                'overwrite': False,
            },
        },
    })
    process.crawl(AmazonReviewsSpider)
    process.start()
    



2024-07-03 17:46:09 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-07-03 17:46:09 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.3.0, Python 3.8.5 (default, Sep  4 2020, 02:22:02) - [Clang 10.0.0 ], pyOpenSSL 24.1.0 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform macOS-10.16-x86_64-i386-64bit
2024-07-03 17:46:09 [scrapy.addons] INFO: Enabled addons:
[]
2024-07-03 17:46:09 [py.warnings] WARNING: /Users/apple/opt/anaconda3/lib/python3.8/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST

ReactorNotRestartable: 

In [54]:
# Открываем файл CSV и загружаем данные в DataFrame
data = pd.read_csv('amazon_reviews.csv')

# Выводим первые несколько строк DataFrame для проверки
data.head()

,title,rating,body,author,date


In [55]:
data.shape

(0, 5)